In [1]:
# IMPORT LIBRARIES
import psycopg2 as pg2
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import snowflake.connector

In [2]:
#CONNECT TO SNOWFLAKE WITH REPORTER CREDENTIALS
conn = snowflake.connector.connect(
    user='emirkabasoglu', #emirkabasoglu
    password='7Bb13287', #7Bb13287
    account='se63477.north-europe.azure',
    warehouse='LOADING_WH',
    database='PROD_ANALYTICS_DB'
    )

In [4]:
#START THE CURSOR AND RUN THE QUERY
cur = conn.cursor()
cur.execute('''
SELECT
Product_Family,
SUM(ARR_EUR) AS TOTAL_ARR
FROM(
SELECT
*,
CASE WHEN t3.ITEM_FAMILY_ID = '2' OR T2.ITEM_ID IN ('8_4','8_5') THEN 'Infinite'
     WHEN  t3.ITEM_FAMILY_ID = '1' OR T2.ITEM_ID IN ('8_1','8_2','8_3') THEN 'People'
     ELSE 'Other'  END AS Product_Family
FROM  PROD_ANALYTICS_DB.DBT_DEPLOY.SUBSCRIPTIONS T2 
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.PRODUCTS T3 ON T3.ITEM_ID =T2.ITEM_ID
WHERE t2.STATUS NOT IN ('future','cancelled' ) ) AS SUB_TABLE
WHERE Product_Family IN ('People','Infinite')
GROUP BY 1

''')


In [5]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)

In [6]:
# ASSIGN TO MAIN DF FOR FUTHER TRANSFORMATION
df_main = df_items_db_fetched

In [8]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [9]:
#Total PEOLE AND INFINITE ARR :
df_main

,PRODUCT_FAMILY,TOTAL_ARR
0,People,9922785.236
1,Infinite,3531081.683


In [ ]:
### Infinite Backlog ###

In [12]:
#START THE CURSOR AND RUN THE QUERY
cur_2 = conn.cursor()
cur_2.execute('''
SELECT
SUM(ARR_EUR) AS TOTAL_INFINTE_FUTURE_ARR
FROM(
SELECT
*,
CASE WHEN t3.ITEM_FAMILY_ID = '2' OR T2.ITEM_ID IN ('8_4','8_5') THEN 'Infinite'
     WHEN  t3.ITEM_FAMILY_ID = '1' OR T2.ITEM_ID IN ('8_1','8_2','8_3') THEN 'People'
     ELSE 'Other'  END AS Product_Family
FROM  PROD_ANALYTICS_DB.DBT_DEPLOY.SUBSCRIPTIONS T2 
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.PRODUCTS T3 ON T3.ITEM_ID =T2.ITEM_ID
WHERE t2.STATUS = 'future') AS SUB_TABLE 
WHERE Product_Family ='Infinite'

''')


In [13]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur_2.fetchall()
cols = []
for i in cur_2.description:
    cols.append(i[0])

df_main_2 = pd.DataFrame(items_db_fetched, columns=cols)

In [14]:
#Infinite Backlog (FUTURE INFINITE ARR) :
df_main_2

,TOTAL_INFINTE_FUTURE_ARR
0,1069636.320


In [ ]:
###Total PEOPLE INFINITE CUSTOMER LEVEL %:

In [16]:
#START THE CURSOR AND RUN THE QUERY
cur_3 = conn.cursor()
cur_3.execute('''

SELECT
COUNT(DISTINCT CUSTOMER_ID) AS TOTAL_CUSTOMER
FROM(
SELECT
SUBSCRIPTION_ID,
CUSTOMER_ID,
CASE WHEN t3.ITEM_FAMILY_ID = '2' OR T2.ITEM_ID IN ('8_4','8_5') THEN 'Infinite'
     WHEN  t3.ITEM_FAMILY_ID = '1' OR T2.ITEM_ID IN ('8_1','8_2','8_3') THEN 'People'
     ELSE 'Other'  END AS Product_Family
FROM PROD_ANALYTICS_DB.DBT_DEPLOY.SUBSCRIPTIONS T2
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.PRODUCTS T3 ON T3.ITEM_ID =T2.ITEM_ID
WHERE t2.STATUS NOT IN ('future','cancelled' ) ) AS SUB_TABLE
WHERE Product_Family IN ('People','Infinite')
''')


In [17]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur_3.fetchall()
cols = []
for i in cur_3.description:
    cols.append(i[0])

df_main_3 = pd.DataFrame(items_db_fetched, columns=cols)

In [18]:
#TOTAL CUSTOMERS:
df_main_3

,TOTAL_CUSTOMER
0,798


In [22]:
### INFINITE CUSTOMERS  :

In [23]:
#START THE CURSOR AND RUN THE QUERY
cur_4 = conn.cursor()
cur_4.execute('''

SELECT
COUNT(DISTINCT CUSTOMER_ID) AS TOTAL_INFINITE_CUSTOMER
FROM(
SELECT
SUBSCRIPTION_ID,
CUSTOMER_ID,
CASE WHEN t3.ITEM_FAMILY_ID = '2' OR T2.ITEM_ID IN ('8_4','8_5') THEN 'Infinite'
     WHEN  t3.ITEM_FAMILY_ID = '1' OR T2.ITEM_ID IN ('8_1','8_2','8_3') THEN 'People'
     ELSE 'Other'  END AS Product_Family
FROM PROD_ANALYTICS_DB.DBT_DEPLOY.SUBSCRIPTIONS T2
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.PRODUCTS T3 ON T3.ITEM_ID =T2.ITEM_ID
WHERE t2.STATUS NOT IN ('future','cancelled' ) ) AS SUB_TABLE
WHERE Product_Family IN ('Infinite')
''')


In [24]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur_4.fetchall()
cols = []
for i in cur_4.description:
    cols.append(i[0])

df_main_4 = pd.DataFrame(items_db_fetched, columns=cols)

In [26]:
#TOTAL CUSTOMERS:
df_main_4

,TOTAL_INFINITE_CUSTOMER
0,302
